# PyTorch

PyTorch und TensorFlow sind die beiden führenden Deep-Learning-Frameworks, und die Wahl zwischen ihnen hängt oft von den individuellen Anforderungen und Präferenzen ab. PyTorch hat aufgrund seiner Benutzerfreundlichkeit, Flexibilität und der aktiven Community viele Anhänger gefunden und wird in vielen akademischen und industriellen Projekten verwendet.

PyTorch ist ein Python-Framework für maschinelles Lernen, das eine schnelle und flexible Experimentierumgebung bietet und mit einer Reihe von Werkzeugen und Bibliotheken geliefert wird, die den Einstieg erleichtern. Es wurde von Facebook AI Research entwickelt und ist auf der Grundlage der Torch-Bibliothek aufgebaut. PyTorch bietet zwei Hauptfunktionen:

* Tensoren, die wie Numpy-Arrays funktionieren, aber auf der GPU ausgeführt werden.
* Automatische Differenzierung, die eine effiziente und schnelle Berechnung von Gradienten ermöglicht

PyTorch kann über den Paketmanager `pip` installiert werden:
```bash 
pip install torch
```
Anschließend kann es in Python importiert werden:
```python
import torch
```

## Ablauf
Meist besteht der Ablauf beim Deep Learning aus folgenden Schritten:
1. Daten laden und aufbereiten
2. Modell definieren
3. Modell trainieren
4. Modell speichern


Wir wollen daher in diesem Kurs diese Schritte genauer betrachten. Am Ende sind Sie in der Lage ein eigenes Modell zu definieren und auf einem eigenen Datensatz zu trainieren.




## Tensoren
Bevor wir uns mit PyTorch und dem Training von Neuronalen Netzen beschäftigen, wollen wir uns mit Tensoren befassen. Tensoren sind eine Art von Datenstruktur, die verwendet wird, um Vektoren und Matrizen zu speichern. Sie sind ähnlich wie Numpy-Arrays, aber im Gegensatz zu diesen können sie auf einer GPU ausgeführt werden, um die Berechnung zu beschleunigen. Tensoren und die zugehörigen Operationen sind die grundlegenden Bausteine, die verwendet werden, um Ein- und Ausgaben sowie die Parameter eines Modells zu repräsentieren.

Da wir uns bereits mit Numpy und `ndarrays` beschäftigt haben, sollte die Umstellung auf Tensoren nicht allzu schwer fallen. Die meisten Operationen, die auf Numpy-Arrays ausgeführt werden können, können auch auf Tensoren ausgeführt werden. 

### Erstellen von Tensoren
Tensoren können mit der Funktion `torch.tensor()` erstellt werden. Die Funktion `torch.tensor()` akzeptiert Daten wie Listen, Numpy-Arrays und andere Tensoren. Die Funktion `torch.tensor()` hat einen Parameter `dtype`, der den Datentyp der Elemente im Tensor angibt. Wenn der Parameter `dtype` nicht angegeben wird, wird der Datentyp automatisch aus den Daten abgeleitet. Die Funktion `torch.tensor()` hat auch einen Parameter `device`, der angibt, auf welchem Gerät der Tensor erstellt werden soll. Wenn der Parameter `device` nicht angegeben wird, wird der Tensor auf der CPU erstellt. Wird ein CPU Tensor aus einem Numpy-Array erstellt, so teilen sie sich den gleichen Speicherplatz. Änderungen am Tensor wirken sich daher auch auf das Numpy-Array aus und umgekehrt. CPU Tensoren können mit der Methode `numpy()` in Numpy-Arrays umgewandelt werden.

```python
# create a tensor on the CPU from a list
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device='cpu')

# create a tensor from a numpy array
np_array = np.array([[1, 2, 3], [4, 5, 6]])
t2 = torch.from_numpy(np_array)
np_array += 1
print(t2) # tensor([[2, 3, 4], [5, 6, 7]], dtype=torch.int32)

# create a tensor from another tensor
t3 = torch.ones_like(t2) # create a tensor of ones with the same properties as t2

# create a numpy array from a tensor
np_array = t3.numpy()
t3 += 1
print(np_array) # [[2, 2, 2], 
                #  [2, 2, 2]]

# print properties of tensors
print(f'Shape of t1: {t1.shape}')       # 2x3
print(f'Data type of t1: {t1.dtype}')   # float32
print(f'Device of t1: {t1.device}')     # cpu

```

Um einen Tensor auf der GPU zu erstellen, muss der Parameter `device` auf `cuda` gesetzt werden. Tensoren können auch nachträglich auf die GPU verschoben werden, indem die Methode `to()` aufgerufen wird. Sind mehrere GPUs verfügbar, kann mit `cuda:0` die erste GPU, mit `cuda:1` die zweite GPU usw. ausgewählt werden. Beachten Sie, dass das Kopieren von Daten zwischen CPU und GPU Zeit kostet, so dass Sie die Daten nur dann auf die GPU kopieren sollten, wenn Sie sie dort verwenden wollen.


```python
# check if GPU is available
if torch.cuda.is_available():
    print('GPU is available')
    device = torch.device('cuda:0')
else:
    print('GPU is not available')
    device = torch.device('cpu')

# create a tensor on the GPU from a list
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device=device)

# move a tensor to the GPU
t2 = torch.ones_like(t1, device='cpu')
t2 = t2.to(device)
```

### Operationen mit Tensoren
Alle von Numpy bekannten Operatoren lassen sich auch mit Tensoren durchführen. 
Für weitere Informationen zu Tensoren schauen Sie in die [Dokumentation](https://pytorch.org/docs/stable/tensors.html).


 

## 1. Daten laden und aufbereiten
PyTorch bietet zwei Datenprimitive: `torch.utils.data.Dataset` und `torch.utils.data.DataLoader`, mit denen Sie sowohl vorgeladene Datensätze als auch Ihre eigenen Daten verwalten können. 

Ein `Dataset` repräsentiert Ihre Rohdaten und ermöglicht den Zugriff auf einzelne Datenpunkte, während ein `DataLoader` diese Daten für das Training oder die Inferenz vorbereitet, indem er sie in Mini-Batches aufteilt, das Vertauschen der Reihenfolge ermöglicht und das Laden in den Speicher effizient gestaltet. Der `DataLoader` ist eine praktische Schnittstelle, die es erleichtert, mit `Dataset`-Objekten zu arbeiten.

### Dataset
Wie in scikit-learn gibt es in PyTorch eine Reihe von Datensätzen, die Sie direkt verwenden können. Diese Datensätze sind in der `torchvision.datasets`-Klasse enthalten. Es gibt für verschiedene Anwendungsfälle verschiedene Datensätze. Die häufigsten sind `MNIST`, `CIFAR10` und `COCO`. Eine vollständige Liste der verfügbaren Datensätze finden Sie in der [Dokumentation](https://pytorch.org/vision/stable/datasets.html).

#### Vordefinierte Datensätze
Um einen vordefinierten Datensatz zu laden, muss zunächst ein `Dataset`-Objekt des gewünschten Datensatzes erstellt werden. Dieses Objekt kann dann verwendet werden, um auf die Daten zuzugreifen. Die Daten können dann mit der `torchvision.utils.make_grid()`-Funktion visualisiert werden.

```python
# import torchvision
import torchvision

# load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='data/',                    # path where the dataset is stored
    train=True,                      # load the training split. If set to False, load the validation split
    transform=transforms.ToTensor(), # convert images to tensors
    download=True                    # download the dataset if it is not present on disk
)

# access the first image in the dataset
image, label = train_dataset[0]
print(image.shape) # torch.Size([1, 28, 28])
print(label)       # 5

# get the first 10 images in the dataset
images = [image for image, label in [train_dataset[i] for i in range(10)]]

# create a grid of images
grid = torchvision.utils.make_grid(images, nrow=10)
grid = grid.permute(1, 2, 0)    # change the order of the axes from CxHxW to HxWxC
plt.imshow(grid)                # show the grid
plt.show()                      # show the figure
```

#### Transformationen
Häufig müssen die Daten vor dem Training aufbereitet werden. Für diesen Zweck bietet PyTorch eine Vielzahl von Transformationen an. 
Im vorherigen Beispiel wurde beim Erstellen des MNIST Datasets zum Beispiel die Transformation `transforms.ToTensor()` verwendet.
Diese Transformation konvertiert die Daten in Tensoren. Eine vollständige Liste der verfügbaren Transformationen finden Sie in der [Dokumentation](https://pytorch.org/vision/stable/transforms.html).
Die häufigsten Transformationen sind `transforms.ToTensor()`, `transforms.Normalize()` und `transforms.Resize()`.
Um mehrere Transformationen nacheinander anzuwenden, können diese mit der `transforms.Compose()`-Funktion kombiniert werden.


```python

transforms = transforms.Compose([
    transforms.ToTensor(),        # convert images to tensors
    transforms.Normalize(         # normalize the data
        mean=(0.5,),              # mean of the data
        std=(0.5,)                # standard deviation of the data
    )
])

# load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='data/',                    # path where the dataset is stored
    train=True,                      # load the training split. If set to False, load the validation split
    transform=transforms,            # apply the transformations
    download=True                    # download the dataset if it is not present on disk
)
```




#### Eigene Datensätze
Um einen eigenen Datensatz zu laden, muss zunächst ein `Dataset`-Klasse erstellt werden. Diese Klasse muss von der abstrakten Klasse `torch.utils.data.Dataset` erben und die Methoden `__init__`, `__len__` und `__getitem__` überschreiben. Die Methode `__init__` wird aufgerufen, wenn ein `Dataset`-Objekt erstellt wird. Sie wird verwendet, um die Daten zu laden und aufzubereiten. Die Methode `__len__` wird aufgerufen, wenn die Funktion `len()` auf das `Dataset`-Objekt angewendet wird. Sie gibt die Anzahl der Datenpunkte im Datensatz zurück. Die Methode `__getitem__` wird aufgerufen, wenn ein Element des Datensatzes abgerufen wird. Sie gibt den Datenpunkt an der angegebenen Position zurück. 

```python
# import the abstract class Dataset
from torch.utils.data import Dataset

# create a custom dataset class
class MyDataset(Dataset):
    def __init__(self, ...):
        # load and prepare the data
        ...
    
    def __len__(self):
        # return the number of data points
        ...
    
    def __getitem__(self, idx):
        # return the data point at the specified index
        ...
        return images, labels
```


### DataLoader
Der `DataLoader` ist eine praktische Schnittstelle, die es erleichtert, mit `Dataset`-Objekten zu arbeiten. Er kann verwendet werden, um die Daten in Mini-Batches aufzuteilen, die Reihenfolge der Datenpunkte zu vertauschen und die Daten in den Speicher zu laden. 

```python
# import the DataLoader class
from torch.utils.data import DataLoader

# create a DataLoader object
train_dataloader = DataLoader(
    train_dataset,                  # dataset from which to load the data
    batch_size=64,                  # number of data points in each batch
    shuffle=True,                   # shuffle the data
    num_workers=4,                  # number of subprocesses to use for data loading
    drop_last=True                  # drop the last batch if it is smaller than the specified batch size
)

# iterate over the data
for images, labels in train_dataloader:
    # do something with the data
    print(images.shape) # NxCxHxW (N: batch size, C: number of channels, H: height, W: width) 
    ...
```


   
## 2. Modell definieren
Nachdem wir uns mit Tensoren und dem Laden von Daten beschäftigt haben, wollen wir uns nun mit dem Definieren von Modellen beschäftigen.
Neuronale Netze setzen sich aus mehreren Schichten/Modulen zusammen, die Operationen auf den Eingabedaten durchführen.
Der Namespace `torch.nn` enthält alle Bausteine, die Sie zum Aufbau Ihres eigenen neuronalen Netzes benötigen. Jedes Modul in PyTorch ist eine Unterklasse der Klasse `torch.nn.Module`. Ein neuronales Netz ist selbst ein Modul, das aus anderen Modulen (Schichten) besteht. Diese verschachtelte Struktur ermöglicht die einfache Erstellung und Verwaltung komplexer Architekturen.

Um ein eigenes Modell zu definieren, muss eine Klasse erstellt werden, die von der Klasse `torch.nn.Module` erbt. In der `__init__`-Methode der Klasse werden die Schichten des Netzwerks und in der `forward`-Methode die Vorwärtsberechnung des Netzwerks definiert.

```python
# import the nn module
import torch.nn as nn

# create a neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
       # define the layers of the network
        ...

    def forward(self, x):
        # define the forward pass
        ...
```

Um Eingabedaten mit Hilfe des Modelles zu verarbeiten, werden diese dem Modell übergeben. Dadurch wird die `forward`-Methode des Modells zusammen mit einigen Hintergrundoperationen ausgeführt. Rufen Sie `model.forward()` nicht direkt auf! 

```python
# create a model
model = NeuralNetwork()

# move the model to the device
model = model.to(device)

# create a random input tensor
inpt = torch.randn(64, 28*28, device=device, dtype=torch.float32)) 

# get the output of the model
output = model(inpt)
```




### Schichten
Die meisten Modelle bestehen aus mehreren Schichten. PyTorch hat Module in der `torch.nn`-Klasse, die verschiedene Arten von Schichten darstellen. Eine vollständige Liste der verfügbaren Module finden Sie in der [Dokumentation](https://pytorch.org/docs/stable/nn.html). Die häufigsten Module sind `nn.Linear`, `nn.ReLU`, `nn.Sigmoid`, `nn.Softmax`, `nn.Sequential`, `nn.Flatten`, `nn.Dropout`, `nn.MaxPool2d`, `nn.AvgPool2d` und `nn.Conv2d`.





```python
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
```


### Modell-Zoo
Ein großer Vorteil von PyTorch ist der Modell-Zoo. Dieser enthält bekannte vortrainierte Modelle, die Sie direkt verwenden können. Eine vollständige Liste der verfügbaren Modelle finden Sie in der [Dokumentation](https://pytorch.org/vision/stable/models.html). Bekannte Klassifikations-Modelle sind zB `VGG`, `ResNet` und `Inception`.

```python
import torchvision.models as models
from torchvision.models import VGG16_Weights

# load a predefined model (not pretrained)
model = models.vgg16()

# load a pretrained model
model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

```

<!-- ![VGG16](images/pytorch/VGG16.png) -->
<center><img src="images/pytorch/VGG16.png" alt="VGG16" width="500"/></center>

#### Letzte Schicht ersetzen
Soll ein vortrainiertes Netz für eine andere Aufgabe verwendet werden, so muss oft die letzte Schicht ersetzt werden, da sich die Anzahl der Klassen geändert hat. So hat zB der ImageNet Datenstz 1000 Klassen, aber wir wollen nur 10 Klassen klassifizieren.
Dazu muss zunächst überprüft werden, wie die letzte Schicht implementiert wurde. In PyTorch kann der Aufbau eines Netzes mit der `print` Funktion ausgegeben werden. 

```python
# load a pretrained model
model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# print the model
print(model)
```
Dies führt zur Folgenden Ausgabe:
```
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (18): ReLU(inplace=True)
    (19): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (20): ReLU(inplace=True)
    (21): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (22): ReLU(inplace=True)
    (23): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (24): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (25): ReLU(inplace=True)
    (26): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (27): ReLU(inplace=True)
    (28): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (29): ReLU(inplace=True)
    (30): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
  (classifier): Sequential(
    (0): Linear(in_features=25088, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)
```

Die letzte Schicht ist ein `nn.Linear`-Modul mit 4096 Eingängen und 1000 Ausgängen. Um die letzte Schicht zu ersetzen, muss ein neues `nn.Linear`-Modul erstellt werden, das 4096 Eingänge und 10 Ausgänge hat. Anschließend muss das neue Modul an die richtige Stelle im Modell eingefügt werden.
    
```python
# load a pretrained model
model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# replace the last layer
model.classifier[6] = nn.Linear(4096, 10)
```






 
## 3. Optimierung
Haben wir ein Modell definiert, müssen wir dieses anschließend trainieren. Dazu müssen wir zunächst einen Optimierer auswählen. Ein Optimierer ist ein Objekt, das die Parameter des Modells aktualisiert. Die meisten Optimierer verwenden den Gradienten der Verlustfunktion, um die Parameter zu aktualisieren. Der Gradient wird mit Hilfe der automatischen Differenzierung berechnet. PyTorch bietet eine Reihe von Optimierern in der `torch.optim`-Klasse an. Eine vollständige Liste der verfügbaren Optimierer finden Sie in der [Dokumentation](https://pytorch.org/docs/stable/optim.html). Die häufigsten Optimierer sind `optim.SGD` und `optim.Adam`.

```python
import torch.optim as optim

# create an optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-3)
```

Der Optimierer wird mit den Parametern des Modells initialisiert. Die Methode `model.parameters()` gibt eine Liste aller Parameter des Modells zurück. Der Parameter `lr` gibt die Lernrate an. Die Lernrate bestimmt, wie stark die Parameter aktualisiert werden. Eine zu hohe Lernrate kann dazu führen, dass das Modell nicht konvergiert, während eine zu niedrige Lernrate zu einem langsamen Training führt. Die Lernrate ist ein Hyperparameter, der experimentell bestimmt werden muss. Verschiedene Optimierer haben verschiedene Hyperparameter.




### Verlustfunktion
Die Verlustfunktion (auch als Kostenfunktion oder Fehlerfunktion bezeichnet) ist eine mathematische Funktion, die verwendet wird, um die Abweichung zwischen den tatsächlichen Daten und den vorhergesagten Daten zu quantifizieren. Die Verlustfunktion spielt eine entscheidende Rolle im Trainingsprozess eines Modells. PyTorch bietet eine Reihe von Verlustfunktionen in der `torch.nn`-Klasse an. Eine vollständige Liste der verfügbaren Verlustfunktionen finden Sie in der [Dokumentation](https://pytorch.org/docs/stable/nn.html#loss-functions). Die häufigsten Verlustfunktionen sind `nn.MSELoss`, `nn.CrossEntropyLoss` und `nn.NLLLoss`.

```python
# create a loss function
criterion = nn.CrossEntropyLoss()
```



### Training
Das Training geschieht iterativ in einem sogenannten Trainingsloop. In jedem Schritt des Trainingsloops werden die folgenden Schritte ausgeführt:
1. Eingabedaten und zugehörige Labels laden
2. Vorwärtsberechnung durchführen
3. Verlust berechnen
4. Gradienten berechnen
5. Parameter aktualisieren

```python
# set the model to training mode
model.train()

# iterate over the data
for images, labels in train_dataloader:
    # move the data to the device
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    output = model(images)

    # calculate the loss
    loss = criterion(output, labels)

    # zero the gradients
    optimizer.zero_grad()

    # backward pass 
    # calculate the gradient of the loss with respect to the parameters
    loss.backward()

    # update the parameters
    optimizer.step()
```

Der Durchlauf durch die Traingsdaten wird als Epoche bezeichnet. Die Anzahl der Epochen ist ebenfalls ein Hyperparameter. Nach jeder Epoche kann das Modell auf den Validierungsdaten getestet werden. Dies geschieht in einem sogenannten Validierungsloop. In jedem Schritt des Validierungsloops werden die folgenden Schritte ausgeführt:
1. Eingabedaten und zugehörige Labels laden
2. Vorwärtsberechnung durchführen
3. Verlust berechnen
4. Genauigkeit berechnen

```python
# set the model to evaluation mode
model.eval()

# disable gradient calculation
with torch.no_grad():
    # iterate over the data
    for images, labels in val_dataloader:
        # move the data to the device
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        output = model(images)

        # calculate the loss
        loss = criterion(output, labels)

        # calculate the accuracy
        accuracy = (output.argmax(dim=1) == labels).float().mean()
```



### Visualisierung
Um den Trainingsfortschritt zu überwachen, können die Verluste und Genauigkeiten während des Trainings und der Validierung gespeichert und über die Epoche gemittelt werden. Diese Werte können dann mit Hilfe von Diagrammen visualisiert werden. In den vorherigen Übungen haben Sie die Bibliothek `matplotlib` kennengelernt. Diese Bibliothek kann beispielsweise für die Visualisierung der Verluste und Genauigkeiten verwendet werden.

```python
# import matplotlib
import matplotlib.pyplot as plt

# create a figure
fig, ax = plt.subplots()

# plot the losses
ax.plot(train_losses, label='train')
ax.plot(val_losses, label='val')

```


In [14]:
import matplotlib.pyplot as plt

%matplotlib widget

def visualize_data(data:dict):
    """
    Visualize the data in a dictionary
    """

    # clear the current figure
    plt.clf()

    for idx, metric in enumerate(data.keys()):
        plt.subplot(1, 2, idx+1)
        plt.title(metric)
        for phase in data[metric].keys():
            plt.plot(data[metric][phase], label=f'{phase} {metric}')
            plt.legend()
    plt.suptitle('Training Metrics')

 
    


 


#### Tensorboard
Eine weitere Möglichkeit der Visualisierung ist Tensorboard. Tensorboard ist ein Tool zur Visualisierung von Daten, die während des Trainings gesammelt wurden. Es wurde ursprünglich für TensorFlow entwickelt, kann aber auch mit PyTorch verwendet werden. Tensorboard kann mit der Bibliothek `tensorboard` installiert werden:
```bash
pip install tensorboard
```
Anschließend kann es in Python importiert werden:
```python
from torch.utils.tensorboard import SummaryWriter
```

Um Daten in Tensorboard zu visualisieren, müssen diese zunächst in ein `SummaryWriter`-Objekt geschrieben werden. Dieses Objekt kann dann verwendet werden, um die Daten in Tensorboard zu visualisieren. 

```python
# create a SummaryWriter object
writer = SummaryWriter('runs/mnist_experiment_1')

# write the loss to Tensorboard
writer.add_scalar('Loss/train', train_loss, epoch)
```

Neben Skalaren können auch Bilder, Histogramme, Graphen und Texte in Tensorboard visualisiert werden. Eine vollständige Liste der verfügbaren Funktionen finden Sie in der [Dokumentation](https://pytorch.org/docs/stable/tensorboard.html).

Um Tensorboard zu starten, muss der folgende Befehl ausgeführt werden:
```bash
tensorboard --logdir=path_to_runs --port=6006 --bind_all
```
Tensorboard kann dann im Browser unter der Adresse `http://im-kigs.oth-regensburg.de:6006` aufgerufen werden. Da wir alle auf dem selben Server arbeiten müssen wir unterschiedliche Ports verwenden. Verwenden Sie daher die Nummer ihrer Kennung (abc12345) als Port für Tensorboard.

#### Interpretation der Ergebnisse
Die Visualisierungen können verwendet werden, um den Trainingsfortschritt zu überwachen und zu verstehen, wie das Modell funktioniert. Der Fehler und die Genauigkeit auf den Trainingsdaten und den Validierungsdaten sollte sich im Laufe der Zeit ändern. Wenn der Fehler auf den Trainingsdaten sinkt, aber auf den Validierungsdaten steigt, ist das Modell wahrscheinlich überangepasst (overfitting). Wenn der Fehler auf den Trainingsdaten und den Validierungsdaten steigt oder stagniert, ist das Modell wahrscheinlich unterangepasst (underfitting). 
 
<img src="images/pytorch/Under_Overfitting.png" alt="loss" width="500"/>

Im Falle von Underfitting kann das Modell komplexer gestaltet werden, indem zB mehr Schichten oder mehr Neuronen pro Schicht hinzugefügt werden. Im Falle von Overfitting kann das Modell vereinfacht werden oder es können Regularisierungstechniken wie Dropout verwendet werden.

Ebenfalls lässt sich die Wahl der Lernrate einordnen. Wenn die Lernrate zu hoch ist, kann das Training instabil werden, da die Gewichtsaktualisierungen zu groß sind und das Modell möglicherweise über die Minima des Fehlerfunktion springt. Dies kann dazu führen, dass der Fehler auf den Trainingsdaten schnell sinkt, aber auf den Validierungsdaten ansteigt, da das Modell nicht gut generalisiert. Ist die Lernrate zu niedrig, kann das Training sehr langsam sein und in einem lokalen Minima steckenbleiben. In diesem Fall wird der Fehler sowohl auf den Trainingsdaten als auch auf den Validierungsdaten nur langsam sinken, und es kann viel Zeit erfordern, bis das Modell konvergiert.

<img src="images/pytorch/Learning_Rate.png" alt="loss" width="500"/>


## 4. Modell speichern

Ein trainiertes Modell kann mit der `torch.save()`-Funktion gespeichert werden. Diese Funktion speichert die Parameter des Modells in einer Datei. Die Datei kann dann mit der `torch.load()`-Funktion geladen werden. Die Funktion `torch.save()` akzeptiert zwei Parameter: das Modell und den Dateinamen. Der Dateiname sollte die Endung `.pt` oder `.pth` haben.

```python
# save the model
torch.save(model, 'model.pt')

# load the model
model = torch.load('model.pt')
```

Üblicherweise soll nicht das Modell selbst, sondern nur die Parameter gespeichert werden. Dies kann mit dem `state_dict` des Modells erreicht werden. Das `state_dict` ist ein Python-Wörterbuch, das jedem Parameter einen eindeutigen Schlüssel zuordnet. Das `state_dict` kann mit der Methode `model.state_dict()` abgerufen werden. Die Methode `model.load_state_dict()` kann verwendet werden, um das `state_dict` in ein Modell zu laden. Häufig werden neben den Modellparametern auch weitere Informationen wie die Anzahl der Epochen, der Optimierer und die Verlustfunktion gespeichert. Diese Informationen können in einem Python-Wörterbuch gespeichert werden und zusammen mit dem `state_dict` gespeichert werden.


```python
# create a dictionary with the model parameters and additional information
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}

# save the dictionary
torch.save(checkpoint, 'checkpoint.pth')

# load the dictionary
checkpoint = torch.load('checkpoint.pth')

# load the model parameters
model.load_state_dict(checkpoint['model_state_dict'])

# load the optimizer parameters
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# load the loss
loss = checkpoint['loss']

# load the epoch
epoch = checkpoint['epoch']
```





## Aufgaben
1. Laden Sie den MNIST Datensatz und visualisieren Sie die ersten 10 Bilder.
2. Definieren Sie ein Modell mit zwei linearen Schichten und einer ReLU-Aktivierungsfunktion. Trainieren Sie das Modell für 10 Epochen und visualisieren Sie die Verluste und Genauigkeiten während des Trainings und der Validierung.
3. Ändern Sie Ihr Modell in ein CNN. Trainieren Sie das Modell für 10 Epochen und visualisieren Sie die Verluste und Genauigkeiten während des Trainings und der Validierung.
4. Speichern Sie während dem Training das beste Modell. Laden Sie das beste Modell nach dem Training und testen Sie es auf den Validierungsdaten.
5. Schreiben Sie sich eine eigene `Dataset`-Klasse.




In [1]:
# Imports 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm


In [24]:
# define the network

class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        #  define the layers


    def forward(self, x):

        # flatten the input

        # define the forward pass
        
        
        return x




In [26]:
class ConvNet(nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        # define conv layers (feature extractor)

        # define the pooling layer

        # define the fully connected layers (classifier)

        # define the activation function

    def forward(self, x):
        return x

In [19]:
# definte training loop
def train_loop(model, criterion, optimizer, train_dataloader, device):
    # set the model to train mode
    model.train()

    # summaraize the loss and accuracy
    epoch_loss = 0
    epoch_accuracy = 0

    # iterate over the data
    for images, labels in train_dataloader:
       
        '''
            TODO: Complete the training loop
        '''

        # calculate the accuracy
        accuracy = (output.argmax(dim=1) == labels).float().mean()

        # accumulate the loss and accuracy
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()

    num_batches = len(train_dataloader)
    return epoch_loss/num_batches, epoch_accuracy/num_batches


def valid_loop(model, criterion, val_dataloader, device):
    # set the model to evaluation mode
    model.eval()

    # summaraize the loss and accuracy
    epoch_loss = 0
    epoch_accuracy = 0

    # disable gradient calculation
    with torch.no_grad():
        # iterate over the data
        for images, labels in val_dataloader:
            '''
                TODO: Complete the validation loop 
            '''

            # calculate the accuracy
            accuracy = (output.argmax(dim=1) == labels).float().mean()

            # accumulate the loss and accuracy
            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()


    num_batches = len(val_dataloader)
    return epoch_loss/num_batches, epoch_accuracy/num_batches

In [ ]:

# load the MNIST dataset
# create a dataset
train_dataset = None
valid_dataset = None


# create a dataloader
train_loader = None
valid_loader = None


# define the loss function
criterion = None


# define model
model = None


# define hyperparameters
num_epochs = 10
lr = 0.001


# define the optimizer
optimizer = None


# define the device
device = None

# dictionary to store the metrics
metrics = {m: {p:[] for p in ['train', 'val']} for m in ['loss', 'acc']}

# use a nice progress bar to visualize the training progress
pbar = tqdm(range(num_epochs))

# train the model for num_epochs epochs
for ep in pbar:
    # set a description for the progress bar
    pbar.set_description(f'Epoch: {ep+1}/{num_epochs}')

    # call the train and valid loops
    train_loss, train_acc = train_loop(model, criterion, optimizer, train_loader, device)
    val_loss, val_acc     = valid_loop(model, criterion, valid_loader, device)

    # store the metrics
    metrics['loss']['train'].append(train_loss)
    metrics['loss']['val'].append(val_loss)
    metrics['acc']['train'].append(train_acc)
    metrics['acc']['val'].append(val_acc)

    # update the progress bar with the metrics from the current epoch
    pbar.set_postfix({'Train Loss':f'{train_loss:.2f}', 'Val Loss':f'{val_loss:.2f}', 'Train Acc':f'{train_acc:.2f}', 'Val Acc':f'{val_acc:.2f}'})
pbar.close()

# visualize the logged metrics over the whole training process
visualize_data(metrics)
plt.show()


